# ATM746 - Laboratory Component

## lecture 2: Setup and Run WRF Real

Today we will go trough the steps to setup and run WRF in Derecho

### Connect to Derecho trough ssh or Jupyter

### References and further reading

**WRF Users Guide**

https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/

**WRF Technical Document**

https://www2.mmm.ucar.edu/wrf/users/docs/technote/contents.html

### Components of a WRF Real Case
<img src="assets/wps_program_flow.png" width="900">
<p><em>Figure 1: WRF system components. From https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/wps.html</em></p>


### The WRF Preprocessing System (WPS)
The WRF Preprocessing System (WPS) is a set of three programs whose collective role is to prepare input to the real program for real-data simulations. Each program performs one stage of the preparation:

1. geogrid defines model domains and interpolates static geographical data to the grids

2. ungrib extracts meteorological fields from GRIB-formatted files

3. metgrid horizontally interpolates the meteorological fields extracted by ungrib to the model grids defined by geogrid

geogrid and ungrib can be executed in any order, but both need to be run before metgrid

For **geogrid**, we need to have the static data.

- https://www2.mmm.ucar.edu/wrf/users/download/get_sources_wps_geog.html
- in Derecho we don't need to download it, it is available `ls ~wrfhelp/WPS_GEOG/`
- it creates geo_em.d0*.nc files (1 file per domain)

For **ungrib**, we need the meteorological data

- it can be obtained from the web, e.g. https://gdex.ucar.edu/datasets/d083003/
- in Derecho we don't need to download it, it is available `ls /glade/campaign/collections/rda/data/d083003/`
- it creates intermediary files, usually named FILE:*
- it requires you to select the correct variable table (“Vtable”) translator 
- before running ungrib the met files need to be linked using `./link_grib.csh`

you can check the variable table files:

from your WPS dir: `cd /glade/work/$USER/atm746/WPS`

type `ls -l ungrib/Variable_Tables/`

you can take a look at the README file with details about the tables

`vi ungrib/Variable_Tables/README`

For **metgrid**, we need the  `geo_em.d0*.nc` and the intermediate `FILE:*`

- Makes a horizontal interpolation of the extracted parent model data 
- It creates a number of files called met_em.d0*.*


For **real**, real-data initialization program, we need the met_em files `met_em.d0*.*`

- Vertical interpolation of metgrid (meteorological fields and sub-surface soil data) from the vertical coordinate of the met_em (parent model) to that of WRF 
- Creates boundary and initial condition files to feed into the wrf.exe program on the terrain-following WRF vertical grid
- it creates `wrfinput_d0*`, `wrfbdy_d0*`. If SST are updated,`sst_update=1` it also creates `wrflowinp_d0*`




For **wrf**, we need the input files from **real**

- executes the model using the input files from real
- produces output as `wrfout_d0*`

### File Formats

Most files we would need to handle, apart from textfiles, are either GRIB (.grib) or NETCDF (.nc). Sometimes file don't have the extension indicating their format

If you want to know more about these file formats

- GRIB: https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them
- NetCDF: https://docs.unidata.ucar.edu/nug/current/netcdf_introduction.html

## WPS - steps for preprocessing the input files

We can control our selections using the following files, usually

- namelist.wps (dates, domain, projection, among other)
- GEOGRID.TBL (defines each of the fields that will be produced by geogrid)
- METGRID.TBL (how each meteorological field is interpolated)
- Vtable (tells ungrib the codes and levels of variables in the GRIB file)

namelist.wps variables can be found here:

- https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/wps.html#wps-namelist-variables

### Shared parameters

Can be found in the `&share` section of the namelist

Usually, you would only change

- `max_dom` number of domains
- `start_date` and `end_date`: simulation start and end dates in `YYYY-MM-DD_hh:mm:ss` format
- `interval_seconds` depends on the meteorological data used for the boundary conditions, some datasets are only every 6h (21600 seconds)

```
&share
 wrf_core = 'ARW',
 max_dom = 2,
 start_date = '2019-09-04_12:00:00','2019-09-04_12:00:00',
 end_date   = '2019-09-06_00:00:00','2019-09-04_12:00:00',
 interval_seconds = 10800
/
```

### GEOGRID

For geogrid, we use the **&geogrid** section of the namelist

```
&geogrid
 parent_id         =   1,   1,
 parent_grid_ratio =   1,   3,
 i_parent_start    =   1,  53,
 j_parent_start    =   1,  25,
 e_we              =  150, 220,
 e_sn              =  130, 214,
 geog_data_res = 'default','default',
 geog_data_path = '/glade/work/wrfhelp/WPS_GEOG/'
 dx = 30000,
 dy = 30000,
 map_proj = 'lambert',
 ref_lat   =  39.70,
 ref_lon   = -124.20,
 truelat1  =  39.7,
 truelat2  =  39.7,
 stand_lon = -124.2,
/
```

` parent_id ` ID of the parent domain, if there is one

For the image below, the parent ID's are the following

`parent_id = 1, 1, 2, 1`

<img src="assets/wps_domains1.png" width="600">


The following figure shows domain nestings that are allowed and not allowed

<img src="assets/wps_domains2.png" width="600">


`parent_grid_ratio` indicates the nesting ratio relative to the domain's parent

Every nest must be a child of exactly one other nest, with the coarse domain being its own parent

Example: `parent_grid_ratio =   1,   3,` indicates that domain 2 has a resolution 3x higher
- Domain 1 has dx = 30km
- Domain 2 then has dx = 10km

- it is recommended to have a ratio of 3. On occassions, a ratio of 5 can be used. 

`i_parent_start` and `i_parent_start` indicate the x and y (or i and j) coordinates of the lower left corner of the nest in the parent domain

```
 i_parent_start    =   1,  53,
 j_parent_start    =   1,  25,
```

Domain 1 is just 1 because there is no parent, and Domain 2 starts in `i = 53` and `j = 25` of Domain 1.

- keep boundaries away from the borders, usually `i,j >= 20`

- keep boundaries away from steep topography

 <img src="assets/wps_domains3.png" width="500">

`e_we`and `e_sn` are the domain's full west-east and south-north dimensions (or grid spaces)

```
 e_we              =  150, 220,
 e_sn              =  130, 214,
 ```

 e_we and e_sn must be one greater than an integer multiple of the nest's parent_grid_ratio.

 This is to ensure that a nest always starts and ends on a parent grid point.

 (i.e., e_we=n*parent_grid_ratio+1 for some integer n).

 In our case, both (219 and 213 are divisible by 3--our parent grid ratio--)

 - Domains should not be smaller than 100x100 grid spaces. 
 - In small domains the solution is dominated by forcing data

`geog_data_res` indicates the resolution of source data to be used when interpolating static terrestrial data

This is related to the paremeters in GEOGRID.TBL

`geog_data_res = 'default','default',`

in our case we use `default` and this means that for topography (HGT_M) it is using the `topo_gmted2010_30s`

**it is recommended to use data resolution slightly better than your grid size**


From your `WPS`directory, if you check the GEOGRIB.TBL (`vi geogrid/GEOGRID.TBL`) you can see the following

```
name = HGT_M
        priority = 1
        dest_type = continuous
        smooth_option = smth-desmth_special; smooth_passes=1
        fill_missing=0.
        interp_option = gmted2010_30s:average_gcell(4.0)+four_pt+average_4pt
        interp_option =     gtopo_30s:average_gcell(4.0)+four_pt+average_4pt
        interp_option =      gtopo_2m:four_pt
        interp_option =      gtopo_5m:four_pt
        interp_option =     gtopo_10m:four_pt
        interp_option =        lowres:average_gcell(4.0)+four_pt
        interp_option =       default:average_gcell(4.0)+four_pt+average_4pt
        rel_path = gmted2010_30s:topo_gmted2010_30s/
        rel_path =     gtopo_30s:topo_30s/
        rel_path =      gtopo_2m:topo_2m/
        rel_path =      gtopo_5m:topo_5m/
        rel_path =     gtopo_10m:topo_10m/
        rel_path =        lowres:topo_gmted2010_5m/
        rel_path =       default:topo_gmted2010_30s/

```

`geog_data_path` indicates the path (location) where the static geographical data is stored.

`geog_data_path = '/glade/work/wrfhelp/WPS_GEOG/'`

In Derecho, the datasets can be found using:

`ls -l ~wrfhelp/WPS_GEOG/`

`dx` and `dy`are the grid size

This only needs to be set for the parent domain, the other domains are defined using the `parent_grid_ratio`

```
 dx = 30000,
 dy = 30000,
```

The rest of the parameters are for the projection

- lambert (midlatitudes)
- polar (high latitudes)
- mercator (tropics)
- lat/lon (for global domains)

<img src="assets/wps_projections.png" width="500">


 ```
 map_proj = 'lambert',
 ref_lat   =  39.70,
 ref_lon   = -124.20,
 truelat1  =  39.7,
 truelat2  =  39.7,
 stand_lon = -124.2,
 ```

`ref_lat` and `ref_lon` are the lat/lon of the center-point of the coarse domain by default (unless you set `ref_x` and `ref_y`, optional paramters)

`truelat1` true latitude, distances are accurate

`truelat2` only for lambert, if domain is small make it equal to `truelat1`

`stand_lon` longitude that is parallel with the y-axis
- make it equal to `ref_lon`, so the parent domain is centered



Remember:

Projections cause distorion, so check map scale factor (e.g., MAPFRAC_M), values should be as close to 1 as possible

map factor = dx_namelist / dx_real

- if map factor > 1.0, the real dx is smaller than the specified dx. Take it into account for **dt**
- if map factor < 1.0, the real dx is larger than the specified dx. You have a lower resolution


**check notebook plot_domain_map.ipynb to see an example**

### UNGRIB

For ungrib, we use the **&ungrib** section of the namelist

You usually won't change anything here. 

`prefix` can be set to anything we want, this is used for the name of intermediate-format files

This is necessary if you are using more than one sources of GRIB data, when you run ungrib multiple times and don't want to overwrite

```
&ungrib
 out_format = 'WPS',
 prefix = 'FILE',
/
```

### METGRID

For metgrid, we use the **&metgrid** section of the namelist

`fg_name` contains the prefix used in ungrib

Also, usually you don't change this. In case you have multiple intermediate-format files, you add their prefix in `fg_name = 'FILE1','FILE2',...`

```
&metgrid
 fg_name = 'FILE'
/
```

### Once the desired namelist.wps options have been set, you can follow these steps to obtain the input files

1. run geogrid.exe
2. link_grib.csh
3. run ungrib.exe
4. run metgrid.exe

In Derecho we need to do this using batch jobs submitted through PBS (a job scheduler)

Examples of job scritps for different programs can be found in Derecho

`ls ~wrfhelp/BATCH_SCRIPTS/`


For example, to run geogrid we can use the following script

```
#!/bin/bash

### Project name
#PBS -A UNVR0028

### Job name
#PBS -N henao-geogrid

### Wallclock time
#PBS -l walltime=00:30:00

### Queue
#PBS -q main 

### Merge output and error files
#PBS -j oe                    

### Select 1 nodes with 8 CPUs each, for 8 MPI processes 
#PBS -l select=1:ncpus=8:mpiprocs=8  


mpiexec ./geogrid.exe
```

### run geogrid.exe

you can run this typing:

`qsub rungeogrid.sh`

you can check the status of your job using:

`qstat -u $USER`

if it worked you will see one `geo_em.d0*.nc` for each domain

you can also check the log file `tail geogrid.log` and should see

`*** Successful completion of program geogrid.exe ***`

### run linkgrib

in your WPS directory

as we are interested in the AR of nov 2024, we can link only files for that period

`./link_grib.csh /glade/campaign/collections/rda/data/d083003/2024/202411/gdas1.fnl0p25.202411{1*,2*}.f00.grib2`

to see the list of files we are linking you can use:

`ls -l /glade/campaign/collections/rda/data/d083003/2024/202411/gdas1.fnl0p25.202411{1*,2*}.f00.grib2`

see that we are selecting the analysis files, ending in `.f00` and not the forecast files `.f03`, `.f06`, etc. 

if the link works, you should see many files like this `GRIBFILE.*` by typing `ls`

### checking the files

when we want to simulate a special event, it is worth checking the boundary conditions to see if the event is present in the input data

an example of checking grib files using python is found here `check_ic_bc.ipynb` (https://github.com/jjhenao/laboratory_atm746/blob/main/check_ic_bc.ipynb)


### run ungrib.exe

**!!it is a serial code, never use more than 1 cpu**

First, we need to set the variable table appropriate for our met data. We are using FNL analyses from NCEP, so we can use GFS

you can check the availabe ables `ls ungrib/Variable_Tables/`

To link the variable table for our case use the following:

`ln -sf ungrib/Variable_Tables/Vtable.GFS Vtable`

if you type

`ls -l Vtable` should show `Vtable -> ungrib/Variable_Tables/Vtable.GFS`

the `->` indicates that the file Vtable is the same as `ungrib/Variable_Tables/Vtable.GFS`

You can run now ungrib:

`./ungrib.exe >& ungrib.log`

type `tail ungrib.log` and if it is done correctly you shoul see:

```
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!  Successful completion of ungrib.   !
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
```

if you type `ls` you should see `FILE:*` files

### run metgrid.exe

you can run this typing:

`qsub runmetgrid.sh`

you can check the status of your job using:

`qstat -u $USER`

type `tail metgrid.log` and you should see:

`*** Successful completion of program metgrid.exe ***``

met_em files should be created for each domain `met_em.d0*.2024-*.nc`

those met_em files are needed by the `real.exe` program to create the input files to run wrf.

we can check the met_em file info using ncdump, a tool to check .nc files

`ncdump -h met_em.d01.2024-11-19_00:00:00.nc | less`

for running wrf you would need info from here, like the number of levels in the atmosphere and soil

`num_metgrid_levels = 34 ;`

`num_sm_layers = 4 ;`


### at this point, you have successfully preprocessed the data for WRF

As we used global files for IC/BC, the intermediate files are large, we can remove them and keep only the met_em

we can check the total size using `du -ach FILE\:-*`, the result is `38G	total`

remove these files using `rm FILE\:2024-11-*`

if you check the total size of WPS `du -sh WPS/` it should be `1.3G	WPS/`

## Plotting the Domains

If you forwarded X11 correctly, it allows you to run an app in Derecho and display it in your machine.

We can use ncview, which is installed in Derecho

type `module load ncview`

then, from the WPS directory type `ncview geo_em.d01.nc`

if it worked, you can click on `2d vars` and select `HGT_M`, and you should see this

that is the terrain height in meters for domain 1 

<img src="assets/wps_domains4.png" width="500">

Otherwise we can use python or another software to plot the domains

- Check the notebook `plot_domain_map.ipynb` (https://github.com/jjhenao/laboratory_atm746/blob/main/plot_domain_map.ipynb)

### if everything looks fine, we can proceed to run WRF (real.exe and wrf.exe)

### The WRF model - producing the solution
The WRF model is composed two main programs:

1. real.exe, interpolates metgrid data (isobaric) to wrf coordinate (hybrid sigma)

2. wrf.exe, produces the forecast

- namelist.input controls the main selection (https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/namelist_variables.html)


We now copy a precompiled WRF, as we did for WPS

check available precompiled programs 

`ls -l ~wrfhelp/derecho_pre_compiled_code/`

copy the latest one

`cp -r ~wrfhelp/derecho_pre_compiled_code/wrfv4.7.1 WRF` 

go to the WRF directory `cd WRF` and type `ls -l run/*.exe`, you should see the executable binary files

```
run/ndown.exe -> ../main/ndown.exe
run/real.exe -> ../main/real.exe
run/tc.exe -> ../main/tc.exe
run/wrf.exe -> ../main/wrf.exe
```


go to test/em_real `cd test/em_real` and type `pwd` to print the path. Save it somewhere to use it later





### run real.exe

First, lets go to scrath, where we have significant storage, to run the model

`cd $SCRATCH`

create a directory for the class, and one for running the test named `ar_test`

`mkdir -p atm746/ar_test`

lets copy the WRF files here

`cp -f /glade/work/jhenao/atm746/WRF/test/em_real/* .`




#### we now need to edit the namelist.input

Details about it can be found using
- `vi README.namelist`
- https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/namelist_variables.html

in the namelist variables are a “single entry” or “max_dom” entry, where “max_dom” indicates a value is expected for each domain

First, we can edit the time_control section


`&time_control`

```
 &time_control
 run_days                            = 0,
 run_hours                           = 6,
 run_minutes                         = 0,
 run_seconds                         = 0,
 start_year                          = 2024, 2024,
 start_month                         = 11,   11,
 start_day                           = 18,   18,
 start_hour                          = 12,   12,
 end_year                            = 2024, 2024,
 end_month                           = 11,   11,
 end_day                             = 24,   24,
 end_hour                            = 12,   12,
 interval_seconds                    = 21600
 input_from_file                     = .true.,.true.,
 history_interval                    = 60,  60,
 frames_per_outfile                  = 1, 1,
 restart                             = .false.,
 restart_interval                    = 720,
 io_form_history                     = 2
 io_form_restart                     = 2
 io_form_input                       = 2
 io_form_boundary                    = 2
 /
```


`run_hours` `run_minutes` etc. are optional, indicates the simulation length in days, hours, minutes, and seconds. It is ignored by real.exe, use it to run for a few hours in a longer simulation

`start_year` `start_day` etc. are starting times, these values are read by real to produce input files

`end_year` `end_day` etc. are ending times, these values are read by real to produce input files

`interval_seconds` time interval in seconds between incoming data from WPS (e.g. files every 6h are 21600)

`input_from_file` Setting this to .true. for nested domains will allow the real.exe program to create wrfinput_d0* files for the nested domains 

`history_interval` Frequency at which to write data to the history (wrfout) file. Default is in minutes, it can be changed to seconds or hours using _s or _h (e.g., history_interval_s)

`frames_per_outfile` number of history output times bulked into each history file
- it is best to use 1, so postprocessing is easier and for restart when is needed (likely for simulation in systems with wall clock restrictions)
- if `frames_per_outfile` is 3, and `history_interval` is 60, in our example the first file would be for `2024-11-18_12:00:00` and will contain 3 time slices, the next file would be `2024-11-18_15:00:00` and will contain another 3 time slices, and so on

`restart` A logical that determines whether this is a restart run. You can only use this if you have restart files (wrfrst*) from a previous run.

`restart_interval` interval in minutes at which you want restart files to be written.

The domain section

```
 &domains
 time_step                           = 90,
 time_step_fract_num                 = 0,
 time_step_fract_den                 = 1,
 max_dom                             = 2,
 e_we                                = 132, 214
 e_sn                                = 130, 190
 e_vert                              = 48,     48,
 dzbot                               = 30.
 dzstretch_s                         = 1.11
 dzstretch_u                         = 1.10
 p_top_requested                     = 5000,
 num_metgrid_levels                  = 34,
 num_metgrid_soil_levels             = 4,
 dx                                  = 30000,
 dy                                  = 30000,
 grid_id                             = 1,     2,
 parent_id                           = 0,     1,
 i_parent_start                      = 1,     32,
 j_parent_start                      = 1,     43,
 parent_grid_ratio                   = 1,     3,
 parent_time_step_ratio              = 1,     3,
 feedback                            = 1,
 smooth_option                       = 0
 /
 ```

 `time_step` Time step (in seconds) for integration. The WRF recommendation is 6 times dx in km (e.g, if dx = 30 km, dt = 180 s)
 - I usually use around 3-5 times dx, in complex terrain a larger dt will cause instabilities (Courant–Friedrichs–Lewy--CFL--)
 - usually recommended to use a value divisible by 3600 (for hourly output)

 `max_dom` number of domains

 `e_we`and `e_sn` are the domain's full west-east and south-north dimensions (or grid spaces). Need to be consistent with the values used in WPS

 `e_vert` The number of vertical levels in WRF
 - must be the same for all domains
 - Typically between 40 and 60 levels, never below 35 levels
 - A common issue is to increase horizontal resolution but keep vertical resolution low, this is not recommended

`dzbot`, `dzstretch_s`, `dzstretch_u` are for the resolution of vertical coordinate

`p_top_requested` The pressure top (in Pa) to use in the model.
- must be available in the incoming WPS data.
- 50 hpa is the recommended, do not make it closer to the surface than thi (higher pressure)

`num_metgrid_levels` and `num_metgrid_soil_levels` are the vertical levels for atmosphere and soil parameters in the met_em
- we checked these values using ncdump in the WPS

from `dx` to `parent_grid_ratio` are the same as in the WPS

`parent_time_step_ratio` is a ratio for the time step
- recommended to use the same ratios for each domain
- 3:1 is a reasonable value usually

`feedback` if set to 1, feedback data from nest to parent domain (two-way-nesting). Values of the coarse domain are overwritten by the values in the nest domain at the coincident points. If set to 0, one-way-nesting

`smooth_option` use smoothing for the parent domain in the area of the nest if feedback is on.
- recommended to keep this option set to 0.


##### WRF uses a horizontally and vertically staggered grid (Arakawa-C)

`e_vert`, `e_we`, and `e_sn` are staggered values

we can see this better later on, but it is important

for example, u(i,j) v(i,j) and θ(i,j) are not at the same coordinate location

<img src="assets/grid-staggering.png" width="500">

The physics section has many options available, responsible for different components of the physical processes. 
These schemes interact with each other during model simulations to emulate physical processes in the Earth’s atmosphere.

https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/physics.html


<img src="assets/phys_schemes.png" width="700">


```
 &physics
 physics_suite                       = 'CONUS'
 mp_physics                          = -1,    -1,
 cu_physics                          = -1,    -1,
 ra_lw_physics                       = -1,    -1,
 ra_sw_physics                       = -1,    -1,
 bl_pbl_physics                      = -1,    -1,
 sf_sfclay_physics                   = -1,    -1,
 sf_surface_physics                  = -1,    -1,
 radt                                = 15,    15,
 bldt                                = 0,     0,
 cudt                                = 0,     0,
 icloud                              = 1,
 num_land_cat                        = 21,
 sf_urban_physics                    = 0,     0,
 fractional_seaice                   = 0,
 /
 ```

 `physics_suite` to use a combination of schemes that work well together, at least in previous research (optional).
 - when this is selected, the selection parameters are set to -1

 `mp_physics` micrphysics, rain & moist processes

 `cu_physics` convection
 - should be on for domains with dx>=10km
 - should be off for domains with dx<=4km
 - gray zone between 3-10 km (dx <= 4km are usually referred to convection permitting)

 `ra_lw_physics` longwave radiation

 `ra_sw_physics` shortwave radiation

 `bl_pbl_physics` boundary layer --PBL-- (turbulence)
 - usually for dx >= 1 km
 - for dx <= 100 m, use large eddy simulations
 - 100 - 1 km is gray zone or *terra incognita* 

 `sf_sfclay_physics` land surface model (land-surface fluxes of heat and moisture)

`sf_surface_physics` surface layer (surface layer diagnostics, which includes exchange and transfer coefficients)

`radt` frequency of calls to radiation scheme (minutes)
- usually recommended to have 1 min per dx, e.g. if dx=30 km, radt = 30 mins. Radiation is expensive
- I recommend to set it to 15 even for coarser domains

`bldt` and `cudt` are as `radt` but for boundary layer and convection, respectively
- 0 is recommended

there are many more parameters, check https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/namelist_variables.html


the `&fdda` section is for nudging the model. We won't cover this for now

https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/namelist_variables.html#fdda


``` 
 &dynamics
 hybrid_opt                          = 2,
 w_damping                           = 0,
 diff_opt                            = 2,      2,
 km_opt                              = 4,      4,
 diff_6th_opt                        = 0,      0,
 diff_6th_factor                     = 0.12,   0.12,
 base_temp                           = 290.
 damp_opt                            = 3,
 zdamp                               = 5000.,  5000.,
 dampcoef                            = 0.2,    0.2,
 khdif                               = 0,      0,
 kvdif                               = 0,      0,
 non_hydrostatic                     = .true., .true.,
 moist_adv_opt                       = 1,      1,
 scalar_adv_opt                      = 1,      1,
 gwd_opt                             = 1,      0,
 /
```

`hybrid_opt` vertical coordinate option (2 is default)

`w_damping` vertical velocity damping, turn it on (1) if model stability is causing problems

`diff_opt` turbulence and mixing option (set it to 2)

`km_opt` eddy coefficient option (set it to 4)

`diff_6th_opt` 6th order numerical diffusion option



dampcoef=0.2
diff_opt = 2 and km_opt = 4 for every domain
`

w_damping = 1, damp_opt = 3 with dampcoef=0.2
diff_opt = 2 and km_opt = 4 for every domain


The `bdy_control` section is for the boundary conditions


<img src="assets/boundary.png" width="600">

``` 
 &bdy_control
 spec_bdy_width                      = 5,
 specified                           = .true.
 /
```

`spec_bdy_width` The number of rows for specified boundary value nudging

`specified` true or false, needs to be true for domain 1 and false for the nests

By default, 1 grid row is used for specified conditions, and 4 rows for relaxation zone. This can be changed

### Running real.exe

We need to copy the met_em, but we can do a soft link

go to your running directory `cd $SCRATCH/atm746/ar_test` and type

`ln -sf /glade/work/$USER/atm746/WPS/met_em.d0* .`

you should have the met_em files here now `met_em.d0*.*`

copy the batch script to run real

`cp ~wrfhelp/BATCH_SCRIPTS/runreal.sh .` and change it as follows


```
#!/bin/bash

### Project name
#PBS -A UNVR0028

### Job name
#PBS -N henao-real

### Wallclock time
#PBS -l walltime=00:30:00

### Queue
#PBS -q main 

### Merge output and error files
#PBS -j oe                    

### Select 1 nodes with 32 CPUs each, for 32 MPI processes 
#PBS -l select=1:ncpus=32:mpiprocs=32


mpiexec ./real.exe
```

submit the job:

`qsub runreal.sh` 

check status:

`qstat -u $USER`

when it is running, there are rsl.out and rsl.error files, one for each cpu used

you can check status there, for instance `tail rsl.error.0000` when finished should say

`real_em: SUCCESS COMPLETE REAL_EM INIT`





### running wrf.exe

copy the `runreal.sh` into `runwrf.sh` and change it to look as follows

```
#!/bin/bash

### Project name
#PBS -A UNVR0028

### Job name
#PBS -N henao-wrf

### Wallclock time
#PBS -l walltime=02:30:00

### Queue
#PBS -q main 

### Merge output and error files
#PBS -j oe                    

### Select 1 nodes with 64 CPUs each, for 64 MPI processes 
#PBS -l select=1:ncpus=64:mpiprocs=64


mpiexec ./wrf.exe
```

submit using `qsub runwrf.sh `

and check using `qstat -u $USER`


when it is running, there are rsl.out and rsl.error files, one for each cpu used

you can check status there, for instance `tail rsl.error.0000` when finished should say

`wrf: SUCCESS COMPLETE WRF`

and you should see the hourly output files: `wrfout_d*...`

The wrfoutput files don't have an extension but are NetCDF files.
